# Optimization

In [2]:
from IPython.display import Image
import numpy as np
import matplotlib.pyplot as plt
import cvxopt

## What is Optimization and what are optimization problems?

Optimization problems involve finding the best solution from a set of feasible solutions. This could mean maximizing profit, minimizing costs, or achieving the best possible outcome under given constraints. Optimization is used in various fields such as engineering, economics, and logistics to make decisions that lead to the most favorable results.

## Mathematical Representation?
In the context of optimization, the objective function represents the quantity to be optimized. It can be either a maximization function (to maximize) or a minimization function (to minimize). Mathematically, it can be represented as:

Maximize $f(x)$ or Minimize $f(x)$

Here, $x$ represents the vector of decision variables.

Constraints are the conditions that the solution must satisfy. These can be equality constraints or inequality constraints. Equality constraints are represented as $g_i(x) = 0$, and inequality constraints are represented as $g_i(x) \leq 0$

## What are the types of Optimization Problems?

1. Linear Optimization 
   
   If both the objective function and the constraints are linear, the problem is a linear optimization problem. Linear optimization problems can be solved using methods like the simplex method or interior-point methods.

   Example: \
   Maximize $c^Tx$ \
   Subject to $Ax \leq b, x \geq 0$
   
2. Nonlinear Optimization

   When the objective function or constraints involve nonlinear equations, the problem becomes nonlinear optimization. Nonlinear optimization problems are generally more challenging and require iterative methods to find solutions. Methods like gradient descent, Newton's method, or genetic algorithms are used to solve nonlinear optimization problems.

   Example: \
   Maximize $f(x) = x^2 + 3x + 5$ \
   Subject to $g(x) = x^2 - 4 \leq 0$ 

3. Integer optimization
   
   In integer optimization, some or all of the decision variables are required to be integers. Integer optimization problems are commonly encountered in discrete optimization scenarios like combinatorial optimization problems.

   Example: \
   Maximize $c^Tx$ \
   Subject to $Ax \leq b, x \in \mathbb{Z}^n$

## What are the types of Optimization Algorithms?

1. Linear Programming (LP)
2. Quadratic Programming (QP)
3. Nonlinear Programming (NLP)
4. Mixed-Integer Linear Programming (MILP) and Mixed-Integer Nonlinear Programming (MINLP)

## Let's look at a quick example

A farmer has 500 acres of land available to plant two crops: wheat and corn. Each acre of wheat requires 2 units of fertilizer and 3 units of water, while each acre of corn requires 4 units of fertilizer and 2 units of water. The farmer has 1200 units of fertilizer and 800 units of water available. Determine the number of acres to plant for each crop to maximize the total yield.

Write the objective function and solve using `cvxopt`

Objective Function:

Maximize  \
Subject to: 


In [12]:
#Setup the LP problem
# put your code here

G = cvxopt.matrix([[2.,3.,1.],[4.,2.,1.,]])
h = cvxopt.matrix([1200.,800.,500.])
c = cvxopt.matrix([-1.,-1.])

#Solve the LP problem (hint of what's needed below)
sol = cvxopt.solvers.lp(c, G, h)
print("Optimal solution (Wheat acres, Corn acres):", sol['x'])


     pcost       dcost       gap    pres   dres   k/t
 0: -3.6087e+02 -3.6087e+02  2e+01  4e-02  2e-16  1e+00
 1: -3.5011e+02 -3.5010e+02  2e-01  7e-04  2e-15  3e-02
 2: -3.5000e+02 -3.5000e+02  2e-03  7e-06  8e-16  3e-04
 3: -3.5000e+02 -3.5000e+02  2e-05  7e-08  2e-15  3e-06
Optimal solution found.
Optimal solution (Wheat acres, Corn acres): [ 1.00e+02]
[ 2.50e+02]



You are a financial analyst managing an investment portfolio. You have five different stocks in your portfolio, each with varying returns and risk levels. Your goal is to maximize the expected return while minimizing the portfolio risk. The portfolio must satisfy the following constraints:

- The total investment amount is $1,000,000 (must use all)
- Each stock’s investment must be non-negative (you cannot short-sell).
- The portfolio return should be at least 10%.

So the goal is to minimize portfolio risk which involves the calculation of the portfolio's variance, which is a quadratic function. The variance of the portfolio is calculated using the formula 

$$\sigma^2_p = \sum_{i}\sum_{j}x_ix_j\sigma_{ij}$$

where $x_i$ and $x_j$ are the weights of stocks $i$ and $j$ in the portfolio and $\sigma_{ij}$ is the covariance between the returns of the stocks $i$ and $j$. The presence of product terms $x_ix_j$ makes the objective function quadratic.


In [10]:
# Some daily returns for 5 stocks over 10 days
daily_returns = np.array([
    [0.1, -0.2, 0.15, 0.05, -0.05],
    [0.15, 0.1, 0.2, 0.1, -0.1],
    [-0.05, 0.2, -0.1, 0.15, 0.05],
    [0.2, -0.15, 0.1, -0.05, 0.1],
    [-0.1, 0.15, 0.05, 0.2, -0.2],
    [0.05, -0.05, 0.2, -0.1, 0.15],
    [0.1, 0.05, -0.05, 0.15, 0.2],
    [-0.05, 0.1, 0.15, -0.2, -0.15],
    [0.15, -0.1, -0.2, 0.05, 0.1],
    [0.05, 0.15, 0.1, 0.2, -0.05]
])

# Calculate the mean returns
mean_returns = np.mean(daily_returns, axis=0)

# Normalize the returns by subtracting the mean returns
normalized_returns = daily_returns - mean_returns

# Calculate the covariance matrix
covariance_matrix = np.cov(normalized_returns, rowvar=False)

Write the objective function and constraints and solve using `cvxopt`

Thoughts
- have 5 stocks with returns over the last 10 days
- want to maximize the average return by choosing a combination of those stocks that adds up to 1 MM
- want to minimize the risk in our portfolio by minimizing the portfolios return variance
- objective function would ideally be the portfolio risk/sum of the average return?

Objective Function:
$$-(return_{x_i}+1)x_i+\sum_{i}\sum_{j}x_ix_j\sigma_{ij}$$

Minimize  \
Subject to:
$$\sum_{i}x_i = 1000000$$
$$x_1,x_2,x_3,x_4,x_5 \geq 0$$
$$\frac{1}{I}\sum_{i}return_{x_i} \geq 0.1$$


In [53]:
mean_returns

array([0.06 , 0.025, 0.06 , 0.055, 0.005])

In [57]:
# Setup your QP
n = len(mean_returns)
P = cvxopt.matrix(2*covariance_matrix)
q = cvxopt.matrix(0.0, (n,1))
# Constraint is greater than or equal too, if you want
# less than or equal too we need to multiply by negative
G = cvxopt.matrix([
    [-1.0,0.0,0.0,0.0,0.0,-0.06],
    [0.0,-1.0,0.0,0.0,0.0,-0.025],
    [0.0,0.0,-1.0,0.0,0.0,-0.06],
    [0.0,0.0,0.0,-1.0,0.0,-0.055],
    [0.0,0.0,0.0,0.0,-1.0,-0.005]
])
# G2 = np.diag(mean_returns)
# G = cvxopt.matrix(np.concatenate([G1,G2]))

# h1 = np.array([
#     [0],
#     [0],
#     [0],
#     [0],
#     [0],
# ])
# h2 = np.array([
#     [-0.1],
#     [-0.1],
#     [-0.1],
#     [-0.1],
#     [-0.1],
# ])
h = cvxopt.matrix([0,0,0,0,0,-0.1], (6,1))
A = cvxopt.matrix(1.0, (1,n))
b = cvxopt.matrix(1000000.0)
# your code here

# Solve the QP problem
solution = cvxopt.solvers.qp(P, q, G, h, A, b)


     pcost       dcost       gap    pres   dres
 0:  1.7078e+09 -6.3276e+09  8e+09  5e-13  5e+05
 1:  1.5246e+09  7.4960e+08  8e+08  2e-10  4e+04
 2:  1.4931e+09  1.4687e+09  2e+07  7e-11  3e+02
 3:  1.4924e+09  1.4919e+09  4e+05  3e-11  3e+00
 4:  1.4924e+09  1.4924e+09  4e+03  7e-11  3e-02
 5:  1.4924e+09  1.4924e+09  4e+01  7e-11  3e-04
 6:  1.4924e+09  1.4924e+09  4e-01  8e-11  3e-06
 7:  1.4924e+09  1.4924e+09  4e-03  6e-11  3e-08
Optimal solution found.


In [58]:
# This should help you to see the solution once you set it up properly
cvxopt.printing.options['dformat'] = '%.1f'
cvxopt.printing.options['width'] = -1
print(solution['x'])

[305607.8]
[267677.5]
[193264.7]
[ 77053.4]
[156396.7]

